In [1]:
# !coding=utf-8
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing
import sklearn.linear_model as linear_model
import math as math
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def import_data_train_valid(path_data, train_rate = 0):
    if train_rate == 0:
        return np.array(pd.read_csv(path_data)), []
    data = np.array(pd.read_csv(path_data))
    np.random.shuffle(data)
    valid_num = round(data.shape[0]*(1-train_rate))
    return data[valid_num:, :], data[0:valid_num, :]
    
def import_data_test(path_data):
    data = np.array(pd.read_csv(path_data))
    return data

def std_scaler(data):
    # Standardization scaler
    # ? Standardization of train data, but what about validation data or test data.
    # ? Can we use the same variance and mean scaling.
    return preprocessing.StandardScaler().fit(data)

def maxmin_scaler(data):
    return preprocessing.MinMaxScaler().fit(data)

def corr_analysis(data, threshold = 0.3):
    # correlate analysis
    # return column need to be remove.
    list_r = []
    for i in range(data.shape[1]-1):
        if (np.std(data[:, i]) != 0):
            list_r.append(np.corrcoef(data[:, i], data[:, -1])[0, 1])
        else:
            list_r.append(0)
            
    list_r = np.array(list_r)
    list_r = preprocessing.minmax_scale(np.abs(list_r))
    return np.argwhere(list_r < threshold), np.argwhere(list_r >= threshold)[:-1]

In [2]:
# import data
data_t, data_v = import_data_train_valid('./train.csv', 0)

# 初始化值很关键，std设置小一点
# corr  rate_l      batch  epoch    loss  train
# 0.005 0.05-0.01   500                   0.5
# 0.05  0.001       1000   10000+   70    0.5
# 0.1   0.001       1000   10000+   76
# 0.2   0.005       500    100000+  82
# 0.3   0.005       1000   10000+   90
# 0.4   0.005       1000   10000+   100

# Data analysis.
# low_col, hig_col = corr_analysis(data_t, 0.9)
# data_t = np.delete(data_t, hig_col, axis=1)
# data_v = np.delete(data_v, low_col, axis=1)

data_train = data_t[:, :-1]
label_train = data_t[:, -1]

# data_valid = data_v[:, :-1]
# label_valid = data_v[:, -1]

# Data Square Augmentation
# !!!! Error, because the power make data can't use same normalization operation.
# data_tmp = data_train
# data_tmp = np.column_stack((data_tmp, np.square(data_train)))
# data_tmp = np.column_stack((data_tmp, np.power(data_train, 3)))
# data_tmp = np.column_stack((data_tmp, np.power(data_train, 4)))
# data_tmp = np.column_stack((data_tmp, np.power(data_train, 5)))
# data_train = np.column_stack((data_tmp, np.power(data_train, 6)))

# preprocess
scaler_x = std_scaler(data_train)
data_train = scaler_x.transform(data_train)
# data_valid = scaler_x.transform(data_valid)

# scaler_y = std_scaler(label_train.reshape([-1, 1]))
# label_train = scaler_y.transform(label_train.reshape([-1, 1]))
# label_valid = scaler_y.transform(label_valid.reshape([-1, 1]))


## Pytorch Net

In [3]:
import torch as torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # full connection layer
        self.fc1 = nn.Linear(385, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 100)
        self.fc4 = nn.Linear(100, 100)
        self.fc5 = nn.Linear(100, 100)
        self.fc6 = nn.Linear(100, 50)
        self.fc7 = nn.Linear(50, 50)
        self.fc8 = nn.Linear(50, 1)
        
    def forward(self, x):
        # forward network
        r1 = F.relu(self.fc1(x))
        x = F.relu(self.fc2(r1))
        r2 = F.relu(self.fc3(x+r1))
        x = F.relu(self.fc4(r2))
        x = F.relu(self.fc5(x))
        r3 = F.relu(self.fc6(x+r2))
        x = F.relu(self.fc7(r3))
        return self.fc8(x+r3)
    
    def loss(self, input_t, target):
        out = self.forward(input_t)
        criter = nn.MSELoss()
        loss = criter(out, target)
        return loss, loss.data.numpy()
    
    def optim(self, input_t, target, lr):
        optimiter = optim.SGD(self.parameters(), lr = 0.0001)
        loss, _ = self.loss(input_t, target)
        optimiter.zero_grad()
        loss.backward()
        optimiter.step()
        

# initial input and label variable
input_t = Variable(torch.Tensor(data_train))
target =Variable(torch.Tensor(label_train))
    
# calculate loss
# out = net.forward(input_t)
# criter = nn.MSELoss()
# loss = criter(out, target)
# print(scaler_y.inverse_transform(loss.data.numpy()))

# print(net.fc1.bias.grad)


# Net instance
net = Net()

# update with a learning rate
rate_l = 0.0005
epoch = 100000
sepoch = 20
for e in range(epoch):
    # backward(): calculate every trainable variable's grad
    net.optim(input_t, target, rate_l)
    if (e%sepoch == 0):
        _, loss_p = net.loss(input_t, target)
        print("e=%d, loss=%f"%(e, loss_p))
        if (loss_p < 1):
            rate_l = 0.0001
            sepoch = 100

e=0, loss=2711.641357
e=20, loss=2662.885498
e=40, loss=2393.313477
e=60, loss=112.888992
e=80, loss=73.008743
e=100, loss=59.849667
e=120, loss=52.221420
e=140, loss=46.942131
e=160, loss=42.954960
e=180, loss=39.793091
e=200, loss=37.207668
e=220, loss=35.039967
e=240, loss=33.189442
e=260, loss=31.586132
e=280, loss=30.179987
e=300, loss=28.938543
e=320, loss=27.829044
e=340, loss=26.828127
e=360, loss=25.919552
e=380, loss=25.090355
e=400, loss=24.326281
e=420, loss=23.618444
e=440, loss=22.959061
e=460, loss=22.341358
e=480, loss=21.762720
e=500, loss=21.216232
e=520, loss=20.698669
e=540, loss=20.207344
e=560, loss=19.740063
e=580, loss=19.294580
e=600, loss=18.869343
e=620, loss=18.462709
e=640, loss=18.073013
e=660, loss=17.699020
e=680, loss=17.340200
e=700, loss=16.995098
e=720, loss=16.662645
e=740, loss=16.342123
e=760, loss=16.032278
e=780, loss=15.732926
e=800, loss=15.443254
e=820, loss=15.162377
e=840, loss=14.889989
e=860, loss=14.625813
e=880, loss=14.369395
e=900, lo

In [7]:
# test
data_test = import_data_test("./test.csv")

#data_test = np.delete(data_test, low_col, axis=1)
data_test = scaler_x.transform(data_test)
y_ = net.forward(Variable(torch.Tensor(data_test)))

f = open("./predict0.3.csv", "w")
f.write("id,reference\n")
for i in range(y_.shape[0]):
    f.write("%d, %.8f\n"%(i, y_.data[i]))
f.close()

## Sklearn SGDRegressor

In [6]:
rgr = linear_model.LinearRegression()
# rgr = linear_model.SGDRegressor(eta0 = 0.0001, verbose=1, max_iter = 100)
rgr.fit(data_train, np.squeeze(label_train))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [7]:
# average squared loss
print(mean_squared_error(scaler_y.inverse_transform(rgr.predict(data_train)),\
                         scaler_y.inverse_transform(label_train)))

68.37325186156326


In [44]:
# test
data_test = import_data_test("./test.csv")

#data_test = np.delete(data_test, low_col, axis=1)
data_test = scaler_x.transform(data_test)
y_ = scaler_y.inverse_transform(rgr.predict(data_test))

f = open("./predict.csv", "w")
f.write("id,reference\n")
for i in range(y_.shape[0]):
    f.write("%d, %.8f\n"%(i, y_[i]))
f.close()

## My SGD
Following algorithm is my SGD regressor, but it is no well that it can only reach mean squared loss of 67.

In [169]:
# Initial train params
w = np.random.normal(loc=-0.5, scale=0.001, size=data_train.shape[1])
b = label_train.mean()


batch = 5000
#rate_l = 0.06
rate_l = 0.006
epoch = 100000

In [170]:
# train 
losst = 0
lossv = 0
for e in range(epoch):
    
    for bat in range(math.ceil(data_train.shape[0]/batch)):
        front = bat*batch
        end = (bat+1)*batch
        if (end > data_train.shape[0]):
            end = data_train.shape[0]
        
        data_batch = data_train[front:end, :]
        y_ = np.matmul(data_batch, w)+b
        
        dif_y = y_-label_train[front:end]
        dif_w = np.mean(data_batch*dif_y.reshape(dif_y.shape[0], 1), axis=0)
        dif_b = dif_y.mean()
        
        # update
        w = w-dif_w*rate_l
        b = b-dif_b*rate_l
    
    # compute correct in train and valid
    
    # train
    if (e%1000==0):
        y_t = np.matmul(data_train, w)+b
        losst = np.mean(np.square(y_t-label_train))

        # valid
        y_v = np.matmul(data_valid, w)+b
        lossv = np.mean(np.square(y_v-label_valid))

        print("[%d] train loss=%f, valid loss=%f"\
              %(e, losst, lossv))
        
    if e%1000 == 0:
        if lossv < 10:
            rate_l = 0.001
        elif lossv < 30:
            rate_l = 0.002
        elif lossv < 50:
            rate_l = 0.003
        elif lossv < 70:
            rate_l = 0.004
        elif lossv < 90:
            rate_l = 0.005
    
    

[0] train loss=439.035438, valid loss=478.316155
[1000] train loss=68.266391, valid loss=70.480676
[2000] train loss=68.218196, valid loss=70.451167
[3000] train loss=68.207164, valid loss=70.436242
[4000] train loss=68.202488, valid loss=70.428358
[5000] train loss=68.200030, valid loss=70.423896
[6000] train loss=68.198622, valid loss=70.421222
[7000] train loss=68.197777, valid loss=70.419549
[8000] train loss=68.197252, valid loss=70.418471
[9000] train loss=68.196915, valid loss=70.417763
[10000] train loss=68.196692, valid loss=70.417293
[11000] train loss=68.196540, valid loss=70.416979
[12000] train loss=68.196433, valid loss=70.416768
[13000] train loss=68.196356, valid loss=70.416627
[14000] train loss=68.196298, valid loss=70.416533
[15000] train loss=68.196253, valid loss=70.416471
[16000] train loss=68.196217, valid loss=70.416431
[17000] train loss=68.196188, valid loss=70.416405
[18000] train loss=68.196164, valid loss=70.416389
[19000] train loss=68.196143, valid loss=7

In [111]:
data_train.shape

(11250, 375)